
## "Detection and Use of Insider Trading Information" 
 ### Executive Summary:

This project leverages the availability of large amounts of legally reported insider trading data and an Application Programming Interface available from the SEC to perform the following:
1. Develop software to scan the SEC's database for historical and current reports of insider trades. By law, these reports must be filed with SEC within two days of the trade, and the SEC makes them available immediately.
2. Use historical insider trading data together with publicly available historical stock price data to investigate the short-term influence, if any, of instances of insider trading on the underlying stock price.
3. Determine whether it is possible to detect the influence of individual insider trades on the stock price in real-time or on the same day that the trade occurs. If this were possible, it would be of significant value to traders and investors.
4. Develop actionable insights based on these analyses.

This investigation is carried out for 40 Nasdaq-100 stocks drawn from the technology and communication sectors, and for historical data from 2013-Jan-19 to 2024-Mar-06, and in addition, for benchmarking purposes, the Exchange Traded Funds "QQQ" (which tracks the Nasdaq-100) and "XLK" (which tracks the S&P Technology sector).

**Core Methodology:**
We investigate the influence of insider trading by aligning instances of it with anomalies in the price of the associated stock. If a given insider trade is large enough, it should cause a slight anomalous increase in the stock's price that day relative to the performance of similar stocks on the same day. If this is true and in order to detect it, we carefully define what we mean by "the performance of similar stocks on the same day." For a given "target" stock and a given day (historical), we identify three of our 42 stocks/ETFs which are most closely correlated with the target stock on a 120-day rolling basis. We recompute these correlations for each day to account for the dynamic nature of the markets. On the basis of these three "neighbor" stocks' price movements for that day, we use standard machine learning techniques to infer what we expect the target's price movements to be that same day. If the target stock's price deviates substantially from the inferred price, then we call that an anomaly. We investigate whether instances of insider trading coincide with these price anomalies.

**Statistical Findings:**
There are many factors acting on a given stock's price on any given day, and it turns out that the degree of variability in the price data prevents the inferred price movements from matching the actual price movement to a sufficiently high accuracy (an $R^2$ of 0.65 outside of training data). Hence it is difficult or impossible to identify any one "anomaly" as having a high probability of indicating an insider trade. However, statistical testing (binomial test, t-test) shows a statistically significant increase in the likelihood of price movement exceeding inferred price movements on insider trading days. In other words, there is compelling evidence that insider trading does, on average, have an effect on price.

**Practical Outcome:**
Since insider trades legally must appear on the SEC's website within two business days of the trade, one may wonder whether there is any advantage to buying shares in a stock the day that an SEC report appears telling of an insider buy. Again, individual results vary, but we found that on average there is an advantage in purchasing shares of a stock immediately upon the appearance of a report of an insider buy. We compared the returns of such stocks over the three-day period beginning two days after an insider buy against the Nasdaq-100 index for that same period and found statistically compelling evidence that these returns were higher (0.85% per day) than those of the index (0.56% per day) (p-value $4\times 10^{-4}$).

**Conclusion:**
In conclusion, this project on the detection and utilization of insider trading information has yielded valuable insights into its impact on stock prices and potential advantages for investors. While navigating the complexities of financial markets, our project has demonstrated the feasibility of leveraging SEC-reported data and advanced analytical techniques to identify anomalies indicative of insider trading activity. Despite challenges posed by market variability, statistical analyses have highlighted a significant correlation between insider trading events and abnormal price movements, underscoring the relevance of insider trading information in financial decision-making.



==========================================================================================

#### This notebook assume the existence of three files in the current directory:

1. "tickerlist_naz100.csv" --- This is simple list of stock ticker symbols, one per line.

    
2. "insider_buys.csv" --- This file is produced by the notebook `get_insider_buys.ipynb`, which is the main tool for extracting the necessary info from the SEC and processing it for our use. This notbook takes a couple of hours or so to run. A verion of it is included with this distribution, and so **the reader may skip execution of `get_insider_buys.ipynb`.


3. "lrc_inferred.csv" --- This file is produced by `lateral_predictor.ipynb`. A verion of it is included with this distribution, and so **the reader may skip execution of `lateral_predictor.ipynb`.

In [ ]:
# Most of the functions I've written are in this module:
import insider_trade_detector as itd

# Additionally, I'm using `yfinance` (Yahoo Finance API)
import yfinance as yf

# Other imports
import pandas as pd
from datetime import datetime, timedelta
import random
import numpy as np
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats


#### Load the stock ticker symbols and determine the amount of historical data available
First, read in the list of "ticker symbols" which constitute the set of stocks under consideration. These are mostly small-cap software stocks and related. 

Since stocks vary in how far back their available historical price/volume goes, find a the earliest date for which data is availble for all of our stocks. 

*An important data structure which is loaded here is  `ohlcv_all`.  It is a df with a two-level index: ('ticker','date') and columns

    - Open
    - High
    - Low
    - Close
    - Volume


In [ ]:
'''
This chunk of code mainly produces "ohlcv_all", the "Open, High, Low, Close, Volume" date for 
each stock, for as many dates in the past as they all have in common. 

Also produced here and used later are '`start_date` and '`ticker_dates`

If time allows, put this code chunk into a function 

'''
symbol_list = sorted(itd.read_tickerlist_csv('tickerlist_naz100.csv'))
symbol_dict = { sym: yf.Ticker(sym) for sym in symbol_list }

start_date, late_ticker = itd.get_start_date(symbol_list,start_date_pad=100)
start_date.strftime('%Y-%m-%d')
yesterday_date =  (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')

ohlcv_all = pd.concat(
    {sym: symbol_dict[sym].history(sym, start=start_date, end=yesterday_date, actions=False)
     .tz_convert('UTC') for sym in symbol_dict},
    names=['ticker', 'date']
)

# Verify that each ticker has the same date indices.
unique_tickers = ohlcv_all.index.get_level_values('ticker').unique()
first_ticker_dates = ohlcv_all.xs(unique_tickers[0], level='ticker').index
all_dates_same = True
for ticker in unique_tickers:
    ticker_dates = ohlcv_all.xs(ticker, level='ticker').index
    if not ticker_dates.equals(first_ticker_dates):
        all_dates_same = False
        print(f'\t {ticker} has a wrong number of dates')
if all_dates_same:
    print("All tickers have the same Date indices.")
else:
    print("Not all tickers have the same Date indices.")

print(f"{len(symbol_list)} tickers in all. Data starts on {start_date}. ")


#### Next, read the data from a .csv file which was earlier created by ``get_insider_buys.ipynb`

In [ ]:
'''
Next, read-in the pre-processed insider buy data. Thse are from a .csv file
which is created by `get_insider_buys.ipynb`
`'''
insider_buys_df = itd.read_csv_to_df(start_date)
insider_buys_df.head()

#### One more bit of reading to do: The "lateral predictions". 

* For each stock in our set and for each date starting about 6 months after the start of our data availabilty, we've gon through the process of identifying that stock's nearest neighboring stocks, and inferring or "laterally predicting" that stock's log-return close ("lrc") for that day, on the basis of its' neighbors' lrc values. 

* These and related data are saved in a .csv file which is now read into `log_returns_predictions` 

* `log_returns_predictions` is a df with a two-level index: ('ticker','date') and columns
    - lrc
    - lrc_inferred
    - r2_train --- the R^2 value obtained on the training data 
    - r2_test --- the R^2 values obtained on the test data


In [ ]:
log_returns_predictions = pd.read_csv('lrc_inferred.csv')
log_returns_predictions['date'] = pd.to_datetime(log_returns_predictions['date'], utc=True)
log_returns_predictions.set_index(['ticker', 'date'], inplace=True)


In [ ]:
'''
Check how far back our lrc predictions go, and adjust `insider_buys_df` accordingly.
'''
start_date = log_returns_predictions.index.get_level_values('date').min() 
start_date = pd.to_datetime(start_date, utc=True)
end_date='2024-03-06' 
end_date = pd.to_datetime(end_date, utc=True)

insider_buys_df = insider_buys_df[(insider_buys_df['trade_date'] >= start_date) & (insider_buys_df['trade_date'] <= end_date)]
insider_buys_df.reset_index(drop=True, inplace=True)


### Now begin testing

#### First, how well do our "lateral predictions" work?

The quantity we're estimating is `lrc`, the log-return closing price for a given stock on a given day. The estimated value of this is `lrc_inferred`. 

Make a scatterplot  of `lrc` vs `lrc_inferred`, so that `lrc_inferred` is on the x-axis, and `lrc` is on the y-axis.

**ideally we'd see these points all falling on or near the line y=x.**

Since there are over 100,000 (ticker,date) pairs for which we computed lrc-predictions, an actual scatter plot is too crowded. Let's look at a heat map instead:

In [ ]:
# Extract the columns for the x and y data
mask = (log_returns_predictions['lrc'].abs() < 0.04) & (log_returns_predictions['lrc_inferred'].abs() < 0.04)

x = log_returns_predictions[mask]['lrc']
y = log_returns_predictions[mask]['lrc_inferred']
xmin = np.min(x); xmax = np.max(x)
ymin = np.min(y); ymax = np.max(y)

bins = 100
heatmap_data, x_edges, y_edges = np.histogram2d(x, y, bins=bins)

smoothed_heatmap_data = gaussian_filter(heatmap_data, sigma=2)  
plt.figure(figsize=(10, 8))  


ax = sns.heatmap(smoothed_heatmap_data.T, cmap="viridis")
ax.set_aspect('equal')

plt.gca().invert_yaxis()

origin_x = np.searchsorted(x_edges, 0, side="left")
origin_y = np.searchsorted(y_edges, 0, side="left")

# Plot a red dot at the origin
plt.plot(origin_x, origin_y, 'ro', markersize=3)  
num_ticks = 5

x_indices = np.linspace(0, bins, num_ticks, endpoint=False).astype(int)
y_indices = np.linspace(0, bins, num_ticks, endpoint=False).astype(int)

# Generate tick labels (rounded to 4 decimal places)
xtick_labels = [f"{x_edges[idx]:.4f}" for idx in x_indices]
ytick_labels = [f"{y_edges[idx]:.4f}" for idx in y_indices]

# Set ticks and labels
plt.xticks(ticks=x_indices, labels=xtick_labels, rotation=90)
plt.yticks(ticks=y_indices, labels=ytick_labels)



plt.title('2D Histogram as Heatmap')
plt.xlabel('lrc')
plt.ylabel('lrc_inferred')
plt.show()

#### Compare days with insder trading to those without

* Alternatively, instead of plotting all +100,000 (lrc_inferrred, lrc) pairs, let's take a random sample, and make a scatter plot.

* Add the line y=x. Again, if the predictions we perfect, they'd all line up on this line.

* In red, plot the (lrc_inferrred, lrc) pairs for (ticker,date) pairs corresponding to an insider buy. Ideally we'd notice that these red point tend to me above the red line, at least on average. This would signal that, on average, the stock price tends to performs better on days of insider buys.

* The graphics **do not clearly show this tendency.**

In [ ]:
red_pairs = list(zip(insider_buys_df['ticker'], insider_buys_df['trade_date']))
x = log_returns_predictions['lrc_inferred']
x_red = log_returns_predictions.loc[red_pairs]['lrc_inferred']

y = log_returns_predictions['lrc']
y_red = log_returns_predictions.loc[red_pairs]['lrc']

nsample = 1000 
np.random.seed(42)  
sample_indices = np.random.choice(len(x), nsample, replace=False)
x_sample = x.iloc[sample_indices]; 
y_sample = y.iloc[sample_indices]


plt.figure(figsize=(10, 8))  
plt.scatter(x_sample,y_sample, label='log-return, actual vs. inferred', s=10, alpha=0.4)
plt.scatter(x_red, y_red, color='red', s=10, alpha=0.35, label='Insider Buy Points')

# Plotting the ideal line y = x
plt.plot(x, x, color='red', lw=.5, alpha=0.5, label='Ideal')
plt.axis('square')
plt.xlim(-0.06,0.06); plt.ylim(-0.06,0.06)

# Adding labels and title
plt.xlabel('Inferred log-return')
plt.ylabel('Actual log-return')
plt.title('Scatter Plot of (x, y) with Ideal Line y=x')
plt.legend()

# Displaying the plot
plt.grid(True)
plt.show()

### Statistical Tests

The scatter plot does not show a clear tendency for `lrc` values to be higher than predicted in days of insider trading .

But let's do some statistical testing. 

#### How often is `lrc` greater than predicted (y>x) in general, compared with how often that happens on insider buy days.

In [ ]:
s = 'actual LRC exceeded the predicted LRC '
print(f' The {s} {100*sum(x<y)/len(x):.1f}% of the time')
print(f' For days where there was an insider buy, the {s} {100*sum(x_red<y_red)/len(x_red):.1f}% of the time')

Following up in that last line, let's do a **hypothesis test:** If the red points (those corresponding to insider buys) are no more likely to be above the red line than the any of the rest of the point (this is the "null hypothesis"), then how likely is it that we'd see what we just saw? This is the so-called "p-value" (not the same as `bernoulli_p`). A small p-value means that the probability of what we just saw (the 28%) is very unlikely if our null hypothesis is true. 

What is the probabilty that the effect we see in the line above is due to chance? It looks like **there's a statisticaly significant differene.**

In [ ]:
bernoulli_p = sum(x<y)/len(x)
p_all = stats.binomtest(sum(x<y), len(x), p=bernoulli_p, alternative='two-sided').pvalue
p_red = stats.binomtest(np.sum(x_red<y_red), len(x_red), p=bernoulli_p, alternative='two-sided').pvalue
p_all = stats.binomtest(sum(x<y), len(x), p=bernoulli_p, alternative='greater').pvalue
p_red = stats.binomtest(np.sum(x_red<y_red), len(x_red), p=bernoulli_p, alternative='greater').pvalue
print(f' The p-value for all data points is {p_all}, as expected. ')
print(f' The p-value for the red points is {p_red:0.2e}, which means the "null hypothesis" is very unlikely')


** So we can say with confidence that on insider buy days, the actual LRC value is greater than expected.**

#### Another test:

* How does the mean of all the `lrc` values ("y") compare with the mean of the the `lrc` values ("y_red") for insider trade days? Let's compute them and see that they are differnt, let's run  a t-test to quantify our confidence that they are actually different.  

In [ ]:
t_stat, p_value = stats.ttest_ind(y, y_red)
print(f'The mean of all the lrc values is \t \t \t \t {np.mean(y):0.5f}')
print(f'The mean of the lrc values on insider trade days is \t \t {np.mean(y_red):0.5f}')
print(f"The probabilitiy that these means are actually the same is \t {p_value:0.2e}")


The very low p-value indicates that there is statistically significant evidence that the mean of lrc_red is indeed higher than the mean of lrc.


### It's safe to draw the following conclusions from these graphical and statistical investigations:

1. It is generally **not** possible to see the influence of a single insider buy on a particular day, because the effect is smaller then the general variability of the stock price. 
2. There **is** a small but statistically significant increase in a the daily log-return for stocks subject to insider buying.  

### Now is there any practical use for all of this?

**Yes.** 

Over the 40 stocks and the 11 years of data we examined, there were 426 instances of "insiders" buying the stock in the open market. By law, these purchages must be reported to the SEC within two business days. 

So I asked what if a person were to have bought shares in company the day an insider buy report appeared on the SEC website, and held it for 3 business days. Over those 3 days, how would did the stock perform relative to the Nasdaq 100 index (as priced by the ETF "QQQ")? 




### Test for practical utility  of the project

In this block of code, I randomly sample the inider_buys for which I have data. Each of these has a corresponding (ticker,date) pair. For this (ticker,date) pair, look up the preidicted and actual values of the log-return close,  `lrc_inferred` and `lrc` 

The idea that this project examines is that perhaps the `lrc` for a stock with an insider buy that day will be higher on average than the predicted values `lrc_inferred`.

In [ ]:
pt_diff = 3
pp=[]; qq=[]; 
random.seed(42) # We're going to randomly select (with replacement) some instances of insider buys.
plist = []; qlist = []; rlist=[]
for _ in range(1000):
    indx = random.randint(0, len(insider_buys_df) - 1)
    row = insider_buys_df.iloc[indx]
    symb = row['ticker']
    tdate = row['trade_date']

    tdate_index = ticker_dates.get_loc(tdate)+2
    pdate = ticker_dates[tdate_index+2+pt_diff]
    profit = np.log(ohlcv_all.loc[symb,pdate]['Close']/ohlcv_all.loc[symb,tdate]['Close'])
    qrofit = np.log(ohlcv_all.loc['QQQ',pdate]['Close']/ohlcv_all.loc['QQQ',tdate]['Close'])

    plist.append(profit/pt_diff)
    qlist.append(qrofit/pt_diff)


t_stat, p_value = stats.ttest_ind(plist,qlist)
    
print(' Average the daily log-returnsover days for days 3, 4, and 5 after an insider buy :') 
print(f' \t For the stock subject to insider buying: \t {100*np.mean(plist):0.5f}%')
print(f' \t \t \t  For the Nasdaq-100 index: \t {100*np.mean(qlist):0.5f}%')

print(f'\nThe p-value from the test of whether these means might actually be the same is {p_value:0.3e}')


#### This indicates that there is a statistically significant advantage to buying shares in a stock the day that the SEC reports an insider buy of that stock. 